In [ ]:
# 1️⃣ SETUP & IMPORTS
!pip install fuzzywuzzy python-Levenshtein markovify scikit-learn networkx tqdm pyarrow

import os
import pandas as pd
import numpy as np
import sqlite3
import networkx as nx
from fuzzywuzzy import process
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, top_k_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 94.5 MB/s eta 0:00:00


In [ ]:
# 2️⃣ MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')


DATA_DIR = "/content/drive/MyDrive/campus_data"
DB_PATH = os.path.join(DATA_DIR, "campus.db")     # sqlite DB file
EMBED_PARQUET = os.path.join(DATA_DIR, "face_embeddings.parquet")  # embeddings stored separately
MODEL_DIR = DATA_DIR
# -------------------------------------------


Mounted at /content/drive


In [ ]:
# 3️⃣ CSV table mapping
csv_files = {
    "card_swipes": "card_swipes.csv",
    "cctv_frames": "cctv_frames.csv",
    "face_embeddings": "face_embeddings.csv",      # large - we'll store as parquet
    "free_text_notes": "free_text_notes.csv",
    "lab_bookings": "lab_bookings.csv",
    "library_checkouts": "library_checkouts.csv",
    "profiles": "profiles.csv",
    "wifi_logs": "wifi_associations_logs.csv"
}

In [ ]:
# 4️⃣ Initialize DB (create if not exists) and optionally import CSVs into DB.
def initialize_db(import_csvs=True, store_embeddings_as_parquet=True):
    os.makedirs(DATA_DIR, exist_ok=True)
    conn = sqlite3.connect(DB_PATH)
    if import_csvs:
        for table, fname in csv_files.items():
            path = os.path.join(DATA_DIR, fname)
            if not os.path.exists(path):
                print(f"⚠️ Missing file: {path} (skipped)")
                continue
            if table == "face_embeddings" and store_embeddings_as_parquet:
                # load & save as parquet for efficiency, do not put into sqlite
                try:
                    df_emb = pd.read_csv(path)
                    df_emb.to_parquet(EMBED_PARQUET, index=False)
                    print(f"💾 Saved embeddings to {EMBED_PARQUET} ({len(df_emb)} rows)")
                except Exception as e:
                    print("❌ Failed to convert embeddings to parquet:", e)
            else:
                df = pd.read_csv(path)
                # write to sqlite
                df.to_sql(table, conn, if_exists="replace", index=False)
                print(f"✅ Loaded {table} ({len(df)} rows) into DB")
    conn.close()

# Run once (or whenever you want to re-initialize from CSVs)
initialize_db(import_csvs=True)



✅ Loaded card_swipes (8000 rows) into DB
✅ Loaded cctv_frames (7000 rows) into DB
💾 Saved embeddings to /content/drive/MyDrive/campus_data/face_embeddings.parquet (6973 rows)
✅ Loaded free_text_notes (7000 rows) into DB
✅ Loaded lab_bookings (7000 rows) into DB
✅ Loaded library_checkouts (7000 rows) into DB
✅ Loaded profiles (7000 rows) into DB
✅ Loaded wifi_logs (8000 rows) into DB


In [ ]:
# 5️⃣ Helper: load table into DataFrame
def load_table_from_db(table_name):
    conn = sqlite3.connect(DB_PATH)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        df["source"] = table_name
    except Exception as e:
        print(f"⚠️ Error loading {table_name}: {e}")
        df = pd.DataFrame()
    conn.close()
    return df

In [ ]:
# 6️⃣ Load all available tables (except embeddings which we load separately)
dataframes = {}
for table in csv_files.keys():
    if table == "face_embeddings" and os.path.exists(EMBED_PARQUET):
        # we will load embeddings lazily if needed
        print(f"ℹ️ Skipping embeddings table here (use load_embeddings() when needed).")
        continue
    df = load_table_from_db(table)
    if not df.empty:
        dataframes[table] = df

# Optional loader for embeddings when you need them (does not load by default)
def load_embeddings():
    if os.path.exists(EMBED_PARQUET):
        return pd.read_parquet(EMBED_PARQUET)
    # fallback to DB if parquet not present
    return load_table_from_db("face_embeddings")

ℹ️ Skipping embeddings table here (use load_embeddings() when needed).


In [ ]:
# 7️⃣ Column normalization mapping
COLUMN_MAPPING = {
    'student_id': 'entity_id',
    'user_id': 'entity_id',
    'person_id': 'entity_id',
    'card_id': 'card_id',
    'face_id': 'face_id',
    'device_hash': 'device_id',
    'location_id': 'location_id',
    'loc_id': 'location_id',
    'timestamp': 'timestamp',
    'time': 'timestamp',
    'datetime': 'timestamp',
    'date_time': 'timestamp',
    'email_id': 'email',
    'mail': 'email',
    'full_name': 'name',
    'user_name': 'name'
}

def normalize_columns(df):
    df = df.rename(columns={c: COLUMN_MAPPING.get(c.lower(), c.lower()) for c in df.columns})
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    return df

for k, df in list(dataframes.items()):
    df = normalize_columns(df)
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df.drop_duplicates(inplace=True)
    dataframes[k] = df


In [ ]:
# 8️⃣ Build entity graph for resolution
print("Building entity graph for resolution...")
G = nx.Graph()
for name, df in tqdm(dataframes.items()):
    # choose identifier columns that exist in this table
    id_cols = ['entity_id', 'card_id', 'face_id', 'device_id', 'email']
    id_cols = [c for c in id_cols if c in df.columns]
    if not id_cols:
        continue
    for _, row in df.iterrows():
        ids = []
        for c in id_cols:
            val = row.get(c, None)
            if pd.notna(val) and str(val).strip():
                ids.append(str(val).strip())
        # connect all ids from this record
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                G.add_edge(ids[i], ids[j], source=name)

print(f"✅ Graph: nodes={G.number_of_nodes()}, edges={G.number_of_edges()}")

Building entity graph for resolution...


100%|██████████| 7/7 [00:02<00:00,  2.89it/s]

✅ Graph: nodes=30860, edges=62000


In [ ]:
# 9️⃣ Create connected components -> resolved entities
entity_groups = list(nx.connected_components(G))
entity_map = {}
for i, g in enumerate(entity_groups):
    eid = f"E{i+1}"
    for node in g:
        entity_map[node] = eid
print(f"✅ Resolved entities: {len(entity_groups)}")

✅ Resolved entities: 4860


In [ ]:
# 🔟 Apply resolved_entity mapping to each dataframe
for name, df in dataframes.items():
    if 'resolved_entity' not in df.columns:
        df['resolved_entity'] = np.nan
    for key in ['entity_id', 'card_id', 'face_id', 'device_id', 'email']:
        if key in df.columns:
            # map values using entity_map; values not found become NaN
            mapped = df[key].astype(str).map(entity_map)
            df.loc[df['resolved_entity'].isna(), 'resolved_entity'] = mapped
    dataframes[name] = df

print("✅ Applied resolved_entity to dataframes")


✅ Applied resolved_entity to dataframes


In [ ]:
# 1️⃣1️⃣ Merge all sources into master merged_df
merged_df = pd.concat(list(dataframes.values()), ignore_index=True, sort=False)
if 'timestamp' in merged_df.columns:
    merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], errors='coerce')
merged_df = merged_df.sort_values(['resolved_entity', 'timestamp']).reset_index(drop=True)
print(f"✅ merged_df shape: {merged_df.shape}, unique entities: {merged_df['resolved_entity'].nunique()}")



✅ merged_df shape: (51000, 36), unique entities: 4860


In [ ]:
# 1️⃣2️⃣ Fuzzy normalization for name/email (helps ER)
def fuzzy_clean_column(df, column, threshold=90):
    if column not in df.columns:
        return df
    df[column] = df[column].fillna('').astype(str)
    unique_vals = [v for v in pd.unique(df[column]) if v and len(v) > 2]
    canonical = {}
    for val in unique_vals:
        if val in canonical: continue
        matches = process.extract(val, unique_vals, limit=10)
        for m, score in matches:
            if score >= threshold:
                canonical[m] = val
    df[column] = df[column].map(lambda x: canonical.get(x, x))
    return df

merged_df = fuzzy_clean_column(merged_df, 'name', threshold=92)
merged_df = fuzzy_clean_column(merged_df, 'email', threshold=92)
print("✅ Fuzzy normalization done")


✅ Fuzzy normalization done


In [ ]:
# 1️⃣3️⃣ Timeline generation per entity
timeline_df = merged_df.groupby('resolved_entity').apply(
    lambda g: g.sort_values('timestamp')[['timestamp', 'location_id', 'source']].to_dict('records')
).reset_index().rename(columns={0: 'timeline'})
print(f"✅ timelines generated for {len(timeline_df)} entities")



✅ timelines generated for 4860 entities


In [ ]:
# 1️⃣4️⃣ Feature engineering for ML
df_ml = merged_df.dropna(subset=['resolved_entity', 'timestamp']).copy()
df_ml['hour'] = df_ml['timestamp'].dt.hour
df_ml['dayofweek'] = df_ml['timestamp'].dt.dayofweek
df_ml['month'] = df_ml['timestamp'].dt.month
df_ml['location_id'] = df_ml['location_id'].astype(str).fillna("unknown")

le_loc = LabelEncoder()
df_ml['loc_encoded'] = le_loc.fit_transform(df_ml['location_id'])

# Prepare supervised dataset: predict next location (shifted)
X = df_ml[['hour', 'dayofweek', 'month', 'loc_encoded']].copy()
y = df_ml['loc_encoded'].shift(-1).fillna(df_ml['loc_encoded']).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("✅ Train/test prepared")


✅ Train/test prepared


In [ ]:
# 1️⃣5️⃣ Random Forest with class weights & Top-K evaluation
classes = np.unique(y_train)
class_weights = dict(zip(classes, compute_class_weight(class_weight='balanced', classes=classes, y=y_train)))

rf = RandomForestClassifier(n_estimators=120, random_state=42, class_weight=class_weights, n_jobs=-1)
rf.fit(X_train, y_train)

probs = rf.predict_proba(X_test)          # (n_samples, n_classes)
top1_acc = top_k_accuracy_score(y_test, probs, k=1)
top3_acc = top_k_accuracy_score(y_test, probs, k=3)

print(f"\n🎯 RF Top-1 acc: {top1_acc:.3f}, Top-3 acc: {top3_acc:.3f}")
print(classification_report(y_test, rf.predict(X_test)))
importances = pd.Series(rf.feature_importances_, index=X.columns)
print("Feature importances:\n", importances)


🎯 RF Top-1 acc: 0.113, Top-3 acc: 0.408
              precision    recall  f1-score   support

           0       0.04      0.10      0.06       302
           1       0.04      0.12      0.06       312
           2       0.06      0.13      0.08       312
           3       0.05      0.09      0.06       324
           4       0.03      0.07      0.04       304
           5       0.04      0.15      0.07       311
           6       0.06      0.14      0.08       324
           7       0.05      0.10      0.06       297
           8       0.65      0.11      0.19      4357

    accuracy                           0.11      6843
   macro avg       0.11      0.11      0.08      6843
weighted avg       0.43      0.11      0.15      6843

Feature importances:
 hour           0.501323
dayofweek      0.198725
month          0.042760
loc_encoded    0.257192
dtype: float64


In [ ]:
# 1️⃣6️⃣ Build Markov transition matrix
print("\n✅ Building Markov chain transition probabilities...")
# ensure loc_encoded exists in merged_df (use same label encoder mapping)
if 'loc_encoded' not in merged_df.columns:
    # map merged_df location ids using label encoder if possible
    merged_df['location_id'] = merged_df['location_id'].astype(str).fillna("unknown")
    merged_df['loc_encoded'] = le_loc.transform(merged_df['location_id'])

merged_df = merged_df.dropna(subset=['resolved_entity', 'loc_encoded'])
merged_df = merged_df.sort_values(['resolved_entity', 'timestamp']).reset_index(drop=True)
merged_df['next_loc'] = merged_df.groupby('resolved_entity')['loc_encoded'].shift(-1)

transitions_df = merged_df.dropna(subset=['loc_encoded', 'next_loc']).copy()
transitions = transitions_df.groupby(['loc_encoded', 'next_loc']).size().unstack(fill_value=0)
# avoid zero-rows (if any) - drop
transitions = transitions.loc[(transitions.sum(axis=1) > 0), :]
markov_matrix = transitions.div(transitions.sum(axis=1), axis=0)
print("✅ Markov matrix built. Shape:", markov_matrix.shape)

def predict_markov_topk(current_loc_encoded, top_k=3):
    if current_loc_encoded not in markov_matrix.index:
        return []
    probs = markov_matrix.loc[current_loc_encoded].sort_values(ascending=False)
    return probs.head(top_k).index.tolist()



✅ Building Markov chain transition probabilities...
✅ Markov matrix built. Shape: (9, 9)


In [ ]:
# 1️⃣7️⃣ Hybrid predictor (RF probs + Markov)
def hybrid_predict_topk(features_row, current_loc_encoded, top_k=3, alpha=0.5):
    """
    features_row: 1D array-like with RF features (hour, dayofweek, month, loc_encoded)
    current_loc_encoded: int (current loc index)
    alpha: weight for RF vs Markov (0..1) where final_score = alpha*rf + (1-alpha)*markov
    Returns: list of (loc_idx, score) sorted descending
    """
    # RF probabilities (dictionary loc_idx -> prob)
    prob_rf = {}
    rf_proba = rf.predict_proba([features_row])[0]  # shape (n_classes,)
    classes_rf = rf.classes_
    for c_i, p in zip(classes_rf, rf_proba):
        prob_rf[int(c_i)] = float(p)

    # Markov probabilities for current location
    prob_markov = {}
    if current_loc_encoded in markov_matrix.index:
        series = markov_matrix.loc[current_loc_encoded]
        for idx, p in series.items():
            prob_markov[int(idx)] = float(p)

    # Combine scores across all possible classes (union)
    all_classes = set(list(prob_rf.keys()) + list(prob_markov.keys()))
    combined = []
    for c in all_classes:
        r = prob_rf.get(c, 0.0)
        m = prob_markov.get(c, 0.0)
        score = alpha * r + (1 - alpha) * m
        combined.append((c, score))
    combined.sort(key=lambda x: x[1], reverse=True)
    return combined[:top_k]

# example usage:
example_row = X_test.iloc[0].tolist()
current_loc = int(X_test.iloc[0]['loc_encoded'])
print("Hybrid top-3 (loc_idx,score):", hybrid_predict_topk(example_row, current_loc, top_k=3, alpha=0.6))

Hybrid top-3 (loc_idx,score): [(8, 0.388206462031887), (1, 0.12109149777403404), (0, 0.1176710980589481)]


In [ ]:
# 1️⃣8️⃣ Save merged table and models back to DB / disk
# write merged_df to SQL for future quick loading
conn = sqlite3.connect(DB_PATH)
merged_df.to_sql("merged_entity_timeline", conn, if_exists="replace", index=False)
conn.close()

joblib.dump(rf, os.path.join(MODEL_DIR, "rf_model.pkl"))
joblib.dump(markov_matrix, os.path.join(MODEL_DIR, "markov_model.pkl"))
joblib.dump(le_loc, os.path.join(MODEL_DIR, "label_encoder_loc.pkl"))
print("✅ Saved merged table to DB and dumped models to disk")

✅ Saved merged table to DB and dumped models to disk


In [ ]:
# 1️⃣9️⃣ Incremental helper functions (no manual SQL required)
def append_new_csv_rows_to_db(csv_filename):
    """
    Reads CSV at DATA_DIR/csv_filename and appends only the new rows to DB table
    Table name is csv_filename without extension.
    """
    table = os.path.splitext(csv_filename)[0]
    csv_path = os.path.join(DATA_DIR, csv_filename)
    if not os.path.exists(csv_path):
        print("⚠️ CSV not found:", csv_path)
        return
    df_new = pd.read_csv(csv_path)
    df_new = normalize_columns(df_new)
    conn = sqlite3.connect(DB_PATH)
    try:
        df_existing = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    except Exception:
        # table not present: just write
        df_new.to_sql(table, conn, if_exists="replace", index=False)
        print(f"✅ Created table {table} with {len(df_new)} rows")
        conn.close()
        return

    # use a conservative merge to find rows present only in CSV (naive: compare all columns)
    # to reduce false positives, ensure both have same columns and order
    common_cols = [c for c in df_new.columns if c in df_existing.columns]
    if not common_cols:
        # no columns in common -> append all
        df_new.to_sql(table, conn, if_exists="append", index=False)
        print(f"🔄 Appended all rows to {table} (no common columns to diff)")
        conn.close()
        return

    df_new_sub = df_new[common_cols].astype(str).fillna('')
    df_exist_sub = df_existing[common_cols].astype(str).fillna('')

    # mark new rows by concatenated key (fast heuristic)
    df_new_sub['_key'] = df_new_sub.apply(lambda row: '|'.join(row.values), axis=1)
    df_exist_sub['_key'] = df_exist_sub.apply(lambda row: '|'.join(row.values), axis=1)
    new_keys = set(df_new_sub['_key']) - set(df_exist_sub['_key'])
    if not new_keys:
        print(f"✅ No new rows to append for {table}")
        conn.close()
        return
    new_rows = df_new.loc[df_new_sub['_key'].isin(new_keys)]
    # append
    new_rows.to_sql(table, conn, if_exists="append", index=False)
    print(f"🔄 Appended {len(new_rows)} new rows to {table}")
    conn.close()

def sync_all_csvs():
    for fname in csv_files.values():
        append_new_csv_rows_to_db(fname)

# Example: to append new rows after you add/replace CSVs in Drive:
# sync_all_csvs()


In [ ]:
# 2️⃣0️⃣ Lightweight function to update models after new data arrives
def retrain_models_from_db(retrain_rf=True, retrain_markov=True, sample_limit=None):
    # reload merged data from DB
    conn = sqlite3.connect(DB_PATH)
    merged = pd.read_sql_query("SELECT * FROM merged_entity_timeline", conn)
    conn.close()

    # re-normalize and rebuild minimal pipeline (we reuse existing functions)
    merged['timestamp'] = pd.to_datetime(merged['timestamp'], errors='coerce')
    merged = merged.sort_values(['resolved_entity', 'timestamp']).reset_index(drop=True)
    merged = merged.dropna(subset=['resolved_entity', 'timestamp']).copy()
    merged['location_id'] = merged['location_id'].astype(str).fillna("unknown")
    le = LabelEncoder()
    merged['loc_encoded'] = le.fit_transform(merged['location_id'])

    # features
    merged['hour'] = merged['timestamp'].dt.hour
    merged['dayofweek'] = merged['timestamp'].dt.dayofweek
    merged['month'] = merged['timestamp'].dt.month
    Xr = merged[['hour', 'dayofweek', 'month', 'loc_encoded']].copy()
    yr = merged['loc_encoded'].shift(-1).fillna(merged['loc_encoded']).astype(int)

    if sample_limit:
        Xr = Xr.sample(sample_limit, random_state=42)
        yr = yr.loc[Xr.index]

    if retrain_rf:
        Xtr, Xte, ytr, yte = train_test_split(Xr, yr, test_size=0.2, random_state=42)
        classes = np.unique(ytr)
        c_w = dict(zip(classes, compute_class_weight('balanced', classes=classes, y=ytr)))
        model = RandomForestClassifier(n_estimators=120, random_state=42, class_weight=c_w, n_jobs=-1)
        model.fit(Xtr, ytr)
        joblib.dump(model, os.path.join(MODEL_DIR, "rf_model.pkl"))
        print("✅ Retrained & saved RandomForest")

    if retrain_markov:
        merged = merged.sort_values(['resolved_entity', 'timestamp']).reset_index(drop=True)
        merged['next_loc'] = merged.groupby('resolved_entity')['loc_encoded'].shift(-1)
        tdf = merged.dropna(subset=['loc_encoded', 'next_loc']).copy()
        trans = tdf.groupby(['loc_encoded', 'next_loc']).size().unstack(fill_value=0)
        trans = trans.loc[(trans.sum(axis=1) > 0), :]
        mm = trans.div(trans.sum(axis=1), axis=0)
        joblib.dump(mm, os.path.join(MODEL_DIR, "markov_model.pkl"))
        print("✅ Recomputed & saved Markov model")

# Example: retrain_models_from_db()

In [ ]:
# 2️⃣1️⃣ Quick API-like helpers for notebook use
def predict_next_locations_for_row(df_row, top_k=3, alpha=0.6):
    """
    df_row: a pandas Series row containing timestamp and location_id or loc_encoded
    returns: top_k predicted location labels (decoded)
    """
    # build feature vector
    hour = int(df_row['timestamp'].hour) if pd.notna(df_row['timestamp']) else 0
    dow = int(df_row['timestamp'].dayofweek) if pd.notna(df_row['timestamp']) else 0
    month = int(df_row['timestamp'].month) if pd.notna(df_row['timestamp']) else 0
    if 'loc_encoded' in df_row and not pd.isna(df_row['loc_encoded']):
        loc_enc = int(df_row['loc_encoded'])
    else:
        loc_enc = le_loc.transform([str(df_row.get('location_id', 'unknown'))])[0]

    feats = [hour, dow, month, loc_enc]
    combined = hybrid_predict_topk(feats, loc_enc, top_k=top_k, alpha=alpha)
    # decode labels back to location ids
    results = [(le_loc.inverse_transform([int(c)])[0], float(score)) for c, score in combined]
    return results

# Usage example for the most recent event of an entity
sample_entity = merged_df['resolved_entity'].dropna().sample(1).iloc[0]
entity_events = merged_df[merged_df['resolved_entity'] == sample_entity].sort_values('timestamp')
if not entity_events.empty:
    last_row = entity_events.iloc[-1]
    preds = predict_next_locations_for_row(last_row, top_k=3)
    print(f"Predicted next locations for {sample_entity}: {preds}")

# ========= END OF NOTEBOOK =========
print("All done — notebook ready. Use sync_all_csvs() to append new CSV rows and retrain_models_from_db() to update models.")

Predicted next locations for E3815: [('nan', 0.39318837969871506), ('LAB_101', 0.1237904145350501), ('CAF_01', 0.10507817621308815)]
All done — notebook ready. Use sync_all_csvs() to append new CSV rows and retrain_models_from_db() to update models.
